In [2]:
%%capture
!pip install rectools

In [75]:
import copy
import pprint

import pandas as pd
import numpy as np

import requests
from tqdm.notebook import tqdm

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import (
    Precision,
    Accuracy,
    MAP,
    NDCG,
    Serendipity,
    calc_metrics,
)
from rectools.metrics.novelty import NoveltyMetric
from rectools.models import (
    PopularModel,
    RandomModel
)
from rectools.model_selection import TimeRangeSplitter

# Load data

In [4]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [5]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)


kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [6]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [ ]:
#Load data

In [136]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=True) 

In [137]:
train_interactions = Interactions(interactions)

In [131]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [9]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

headtail(interactions)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


# Initialization

In [21]:
N_SPLITS = 3
K = 10
RANDOM_STATE = 32

## Init splitter

In [22]:
# валидируем на 3 периодах по неделе
n_splits = N_SPLITS

cv = TimeRangeSplitter(
    test_size="7D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

## Init models

In [57]:
models = {
    'random_model': RandomModel(random_state=RANDOM_STATE),
    'popular': PopularModel()
}

## Init metrics

In [58]:
# classifier
precision_1 = Precision(k=1)
precision_5 = Precision(k=5)
precision = Precision(k=K)
accuracy_1 = Accuracy(k=1)
accuracy_5 = Accuracy(k=5)
accuracy = Accuracy(k=K)

#ranking
map_k = MAP(k=K, divide_by_k=True)
ndcg = NDCG(k=K, log_base=3)

#beyond accuracy
serendipity = Serendipity(k=K)
novelty = NoveltyMetric(k=K)

In [59]:
metrics = {
    "precision@1": precision_1,
    "accuracy@1": accuracy_1,
    "precision@5": precision_5,
    "accuracy@5": accuracy_5,
    "precision@k": precision,
    "accuracy@k": accuracy,
    "map": map_k,
    "ndcg": ndcg,
    "serendipity": serendipity,
    "novelty": novelty,
}



# Расчёт метрик

In [107]:
class RecoService:
    def __init__(self, 
                 interactions: pd.DataFrame,
                 models: dict,
                 metrics: dict,
                 splitter: TimeRangeSplitter,
                 k: int,
                 n_splits: int = N_SPLITS    
                ):
        self.interactions = interactions
        
        self.models = models
        self.metrics = metrics
        
        self.splitter = splitter
        self.n_splits = n_splits
        
        self.k = k
        
        
    def train(self):
        results, last_models = list(), dict()
        
        cv = self.splitter.split(self.interactions)
        
        for train_ids, test_ids, fold_info in tqdm((cv), total=self.n_splits):
            print(f"\n==================== Fold {fold_info['i_split']}")
            print(fold_info)
            
            df_train = self.interactions.df.iloc[train_ids]
            dataset = Dataset.construct(df_train)

            df_test = self.interactions.df.iloc[test_ids][Columns.UserItem]
            test_users = np.unique(df_test[Columns.User])

            # Catalog is set of items that we recommend.
            # Sometimes we recommend not all items from train.
            catalog = df_train[Columns.Item].unique()

            for model_name, model in self.models.items():
                model = copy.deepcopy(model)
                model.fit(dataset)
                
                recos = model.recommend(
                    users=test_users,
                    dataset=dataset,
                    k=self.k,
                    filter_viewed=True,
                )
                
                metric_values = calc_metrics(
                    self.metrics,
                    reco=recos,
                    interactions=df_test,
                    prev_interactions=df_train,
                    catalog=catalog,
                )
                res = {"fold": fold_info["i_split"], "model": model_name}
                res.update(metric_values)
                results.append(res)
                last_models[model_name] = model
        
        pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg("mean")
        
        return {
                'results': pivot_results,
                'models': last_models
        }
               
    

In [138]:
reco = RecoService(
    interactions = train_interactions,
    models = models,
    metrics = metrics,
    splitter = cv,
    k = K
)

In [139]:
%%time

output = reco.train()

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00'), 'end': Timestamp('2021-08-09 00:00:00')}

==================== Fold 1
{'i_split': 1, 'start': Timestamp('2021-08-09 00:00:00'), 'end': Timestamp('2021-08-16 00:00:00')}

==================== Fold 2
{'i_split': 2, 'start': Timestamp('2021-08-16 00:00:00'), 'end': Timestamp('2021-08-23 00:00:00')}
CPU times: user 57.9 s, sys: 2.08 s, total: 60 s
Wall time: 1min


In [140]:
(
    output['results'].style
    .highlight_min(color='lightcoral', axis=0)
    .highlight_max(color='lightgreen', axis=0)
)


,precision@1,accuracy@1,precision@5,accuracy@5,precision@k,accuracy@k,ndcg,map,novelty,serendipity
model,,,,,,,,,,
random_model,0.000221,0.999760,0.000202,0.999500,0.000193,0.999176,0.000200,0.000060,15.613009,0.000007
popular,0.076432,0.999770,0.052402,0.999534,0.033903,0.999220,0.043084,0.016591,3.713390,0.000002


# Визуальный анализ

In [132]:
item_interactions = interactions.df.groupby('item_id')['user_id'].count().to_frame().rename(columns={'user_id': 'count_interactions'}).reset_index()
items = items.merge(item_interactions, on='item_id', how='left')
items['count_interactions'] = items['count_interactions'].fillna(0)

In [185]:
class VizualizeModel:
    def __init__(
        self,
        model,
        dataset,
        item_data: pd.DataFrame,
        k: int = K,
        item_columns: list = ['item_id', 'content_type', 'title', 'genres', 'count_interactions']
    ):
        self.model = model
        self.k = k
        
        self.interactions = dataset
        self.dataset = Dataset.construct(dataset)
        self.item_data = item_data[item_columns]
    
    def show_recos(self, user_ids: list):
        recos = self.model.recommend(
                    users=user_ids,
                    dataset=self.dataset,
                    k=self.k,
                    filter_viewed=True,
                )
        
        for user in user_ids:
            print(f"\n==================== User {user}")
            watched_ids = self.interactions[self.interactions['user_id']==user]['item_id'].to_frame()
            reco_ids = recos[recos['user_id']==user]['item_id'].to_frame()
            
            history = watched_ids.merge(self.item_data, on='item_id', how='left')
            recommended = reco_ids.merge(self.item_data, on='item_id', how='left')
            
            print('----------History----------')
            display(history)
            
            print('----------Recos----------')
            display(recommended)
            
            print()
    

In [186]:
dataset = Dataset.construct(interactions)

model = PopularModel()
model.fit(dataset)

In [187]:
vizualizer = VizualizeModel(model=model, dataset=interactions, item_data=items)

In [188]:
vizualizer.show_recos([666262, 672861, 955527])


==================== User 666262
----------History----------


,item_id,content_type,title,genres,count_interactions
0,7957,film,Последний викинг,"боевики, историческое, приключения",746.0
1,4785,film,Робин Гуд: Начало,"боевики, триллеры, приключения",485.0
2,12981,film,Томирис,"боевики, драмы, историческое, военные",10370.0


----------Recos----------


,item_id,content_type,title,genres,count_interactions
0,10440,series,Хрустальный,"триллеры, детективы",202457.0
1,15297,series,Клиника счастья,"драмы, мелодрамы",193123.0
2,9728,film,Гнев человеческий,"боевики, триллеры",132865.0
3,13865,film,Девятаев,"драмы, военные, приключения",122119.0
4,4151,series,Секреты семейной жизни,комедии,91167.0
5,3734,film,Прабабушка легкого поведения,комедии,74803.0
6,2657,series,Подслушано,"драмы, триллеры",68581.0
7,4880,series,Афера,комедии,55043.0
8,142,film,Маша,"драмы, триллеры",45367.0
9,6809,film,Дуров,документальное,40372.0




==================== User 672861
----------History----------


...

,item_id,content_type,title,genres,count_interactions
0,10440,series,Хрустальный,"триллеры, детективы",202457.0
1,15297,series,Клиника счастья,"драмы, мелодрамы",193123.0
2,9728,film,Гнев человеческий,"боевики, триллеры",132865.0
3,13865,film,Девятаев,"драмы, военные, приключения",122119.0
4,4151,series,Секреты семейной жизни,комедии,91167.0
5,3734,film,Прабабушка легкого поведения,комедии,74803.0
6,2657,series,Подслушано,"драмы, триллеры",68581.0
7,4880,series,Афера,комедии,55043.0
8,142,film,Маша,"драмы, триллеры",45367.0
9,6809,film,Дуров,документальное,40372.0




==================== User 955527
----------History----------


,item_id,content_type,title,genres,count_interactions
0,1183,film,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587.0
1,13371,film,Пеле: Рождение легенды,"драмы, спорт, биография",945.0
2,4725,film,Лобановский навсегда,"спорт, биография, документальное",683.0
3,1238,film,Диего Марадона,"спорт, биография, документальное",691.0


----------Recos----------


,item_id,content_type,title,genres,count_interactions
0,10440,series,Хрустальный,"триллеры, детективы",202457.0
1,15297,series,Клиника счастья,"драмы, мелодрамы",193123.0
2,9728,film,Гнев человеческий,"боевики, триллеры",132865.0
3,13865,film,Девятаев,"драмы, военные, приключения",122119.0
4,4151,series,Секреты семейной жизни,комедии,91167.0
5,3734,film,Прабабушка легкого поведения,комедии,74803.0
6,2657,series,Подслушано,"драмы, триллеры",68581.0
7,4880,series,Афера,комедии,55043.0
8,142,film,Маша,"драмы, триллеры",45367.0
9,6809,film,Дуров,документальное,40372.0


In [189]:
dataset = Dataset.construct(interactions)
random_model = RandomModel(random_state=RANDOM_STATE)
random_model.fit(dataset)

In [190]:
vizualizer = VizualizeModel(model=random_model, dataset=interactions, item_data=items)

In [191]:
vizualizer.show_recos([666262, 672861, 955527])


==================== User 666262
----------History----------


,item_id,content_type,title,genres,count_interactions
0,7957,film,Последний викинг,"боевики, историческое, приключения",746.0
1,4785,film,Робин Гуд: Начало,"боевики, триллеры, приключения",485.0
2,12981,film,Томирис,"боевики, драмы, историческое, военные",10370.0


----------Recos----------


,item_id,content_type,title,genres,count_interactions
0,10101,series,Возвращение Будулая,мелодрамы,99.0
1,619,film,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",1.0
2,12618,film,Пропавшая грамота,"фэнтези, комедии",51.0
3,5967,series,Братья вне игры,"драмы, спорт",262.0
4,4041,film,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",19.0
5,5701,film,Алые паруса: Новая история,"комедии, мелодрамы",4.0
6,9738,series,Женщина в беде 3,"детективы, мелодрамы",2.0
7,15247,film,Гордость и предубеждение,"драмы, мелодрамы",150.0
8,10004,film,Болванчики,"мультфильм, приключения, комедии",51.0
9,2816,film,Избави нас от лукавого,"ужасы, триллеры, детективы",1370.0




==================== User 672861
----------History----------


...

,item_id,content_type,title,genres,count_interactions
0,9457,film,Комната (жестовым языком),"драмы, зарубежные, триллеры",5.0
1,15730,series,Твое подтянутое тело,фитнес,2.0
2,473,series,Кто такой Букабу?,"развлекательные, для детей, документальное",15.0
3,12736,film,Палач,"драмы, зарубежные, комедии",3.0
4,3927,film,Помни меня,"драмы, мелодрамы",2982.0
5,3300,film,Антилопа Гну. Южная Африка,документальное,8.0
6,5334,series,Boys and Toys,no_genre,3.0
7,14273,film,Влюбленный скорпион,"драмы, зарубежные, спорт, триллеры, мелодрамы",2.0
8,3087,series,Жуки - караоке,no_genre,1.0
9,4416,film,Питер,"фэнтези, приключения",33.0




==================== User 955527
----------History----------


,item_id,content_type,title,genres,count_interactions
0,1183,film,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587.0
1,13371,film,Пеле: Рождение легенды,"драмы, спорт, биография",945.0
2,4725,film,Лобановский навсегда,"спорт, биография, документальное",683.0
3,1238,film,Диего Марадона,"спорт, биография, документальное",691.0


----------Recos----------


,item_id,content_type,title,genres,count_interactions
0,496,series,Воскресший Эртугрул,"боевики, драмы, приключения",6167.0
1,4205,series,Дело гастронома №1 (Операция Беркут),"драмы, русские",1.0
2,10822,film,Она защищает Родину,"драмы, советские, военные",2.0
3,10914,film,Великолепная,"зарубежные, комедии, мелодрамы",3.0
4,3999,film,Джиперс криперс,"ужасы, триллеры",648.0
5,15756,film,Ремнант: Всё ещё вижу тебя (жестовым языком),"фантастика, зарубежные, триллеры",2.0
6,14961,film,Битва за Землю,"боевики, ужасы, фантастика, триллеры",2032.0
7,13734,film,Сексуальный массаж и Фантазии,для взрослых,31.0
8,3407,film,Черный капитан,"боевики, русские, военные",1.0
9,14614,film,Настя,"мелодрамы, комедии",2.0
